In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from time import time
import numpy as np
from pandas import read_json, DataFrame
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [2]:
df = pd.read_pickle('data/jokes.df.pickle')
print(df.shape)
df.head()

(177437, 8)


,body,id,score,title,joke,tokenlist,mean_w2v_jokes,d2v
126934,On me,34dwcg,0.0,Go down,Go down On me,[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.000717725, -0.00015209, -0.00122301, -0.000..."
132342,Yet.,31igy0,6.0,There's no I in you.,There's no I in you. Yet.,[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[4.78718e-05, -0.000965138, 0.00084987, 0.0007..."
79979,,3wib1b,0.0,Me,Me,[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.00134806, 0.00157046, -0.00121259, 0.001119..."
127890,...I can't even...,33tkg8,1.0,"What is up with 1, 3, 5, 7, 9?","What is up with 1, 3, 5, 7, 9? ...I can't even...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.000789565, 0.00076308, -0.000643772, 0.0008..."
95909,,3njwft,3.0,You,You,[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[-0.000728371, -0.000511831, -0.000967893, -0...."


In [3]:
# added column which classifies one of two groups - is our threshold a score of 15?
df['group'] = 0
df['group'][df['score'] >= 15] = 1
df = df.sort_values('score',ascending=False)
df.head()

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,body,id,score,title,joke,tokenlist,mean_w2v_jokes,d2v,group
642,On the condition he gets to install windows.\n...,5tn84z,48526.0,Breaking News: Bill Gates has agreed to pay fo...,Breaking News: Bill Gates has agreed to pay fo...,"[agreed, breaking, condition, gates, gets, ins...","[1.21590717272325, 1.0461413088169964, -0.5181...","[0.288531, -0.286875, 0.0270473, 0.289043, 0.0...",1
20068,/r/jokes\n\n😜,4xk1en,45500.0,I found a place where the recycling rate is 98...,I found a place where the recycling rate is 98...,"[98, jokes, place, rate, recycling]","[-0.05827733129262924, 0.8892425060272217, 0.1...","[-0.293651, -0.0366122, -0.112096, -0.0572681,...",1
3042,"But its a silly comparison really, its like co...",5s9jog,39570.0,Steve jobs would have been a better president ...,Steve jobs would have been a better president ...,"[apples, better, comparing, comparison, donald...","[0.8610058166086674, 1.2698077009274409, 0.216...","[0.198689, 0.190752, 0.437797, 0.195528, -0.13...",1
22210,We went and had some drinks. Cool guy. Wants t...,4wgall,36421.0,My girlfriend told me to take the spider out i...,My girlfriend told me to take the spider out i...,"[cool, developer, drinks, girlfriend, guy, ins...","[0.6388854711937407, 0.8514862634862462, -0.70...","[-0.179096, 0.168665, -0.291679, 0.0504523, -0...",1
35801,Please don't upvote. Her strap-on is huge.,4pj3q3,35772.0,"For every upvote this gets, my girlfriend and ...","For every upvote this gets, my girlfriend and ...","[anal, don, gets, girlfriend, huge, sex, strap...","[-0.3656227954409339, 0.40265238970856776, -0....","[-0.239757, -0.228843, -0.141935, -0.172717, -...",1


In [4]:
# For a single-input model with 2 classes (binary classification):
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=300))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
t0 = time()
# with open('data/jokes.df.pickle', 'rb') as pickle_file:
#     df = pickle.load(pickle_file)
# print("done in %0.3fs" % (time() - t0))
print('Flattening {} d2v lists ...'.format(df.d2v.values.shape[0]))
X = np.reshape([r for r in df.d2v.values], (df.d2v.values.shape[0], 300))
y = pd.cut(df.score, [-1, 15, 50000], labels=[0.,1.])
print(df.score.describe())
print("done in %0.3fs" % (time() - t0))

print('Splitting into a training and testing set ...')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)
print("done in %0.3fs" % (time() - t0))

# Train the model, iterating on the data in batches of 32 samples
model.fit(X_train, y_train, epochs=10, batch_size=32)
score_test = model.evaluate(X_test, y_test, batch_size=32)

Flattening 177437 d2v lists ...
count    177437.000000
mean        118.042719
std         943.342023
min           0.000000
25%           0.000000
50%           3.000000
75%          15.000000
max       48526.000000
Name: score, dtype: float64
done in 0.865s
Splitting into a training and testing set ...
done in 1.267s
Epoch 1/10
133077/133077 [==============================] - 10s 76us/step - loss: 0.5468 - acc: 0.7506
Epoch 2/10
133077/133077 [==============================] - 9s 68us/step - loss: 0.5338 - acc: 0.7552
Epoch 3/10
133077/133077 [==============================] - 8s 58us/step - loss: 0.5288 - acc: 0.7588
Epoch 4/10
133077/133077 [==============================] - 7s 56us/step - loss: 0.5244 - acc: 0.7613
Epoch 5/10
133077/133077 [==============================] - 7s 56us/step - loss: 0.5201 - acc: 0.7642
Epoch 6/10
133077/133077 [==============================] - 7s 55us/step - loss: 0.5168 - acc: 0.7662
Epoch 7/10
133077/133077 [==============================] - 7s 56us

In [6]:
score_train = model.evaluate(X_train, y_train, batch_size=32)

133077/133077 [==============================] - 5s 39us/step


In [7]:
threshold = 0.5

print ("Train Results\n")
y_pred_train = np.squeeze(model.predict(X_train))
print(classification_report(y_train, y_pred_train > threshold))
print(confusion_matrix(y_train, y_pred_train > threshold))
# score_train = model.evaluate(X_train, y_train, batch_size=32)
# print(score_train)

print ("Test Results\n")
y_pred_test = np.squeeze(model.predict(X_test))
print(classification_report(y_test, y_pred_test > threshold))
print(confusion_matrix(y_test, y_pred_test > threshold))

Train Results

             precision    recall  f1-score   support

        0.0       0.79      0.97      0.87     99951
        1.0       0.69      0.20      0.31     33126

avg / total       0.76      0.78      0.73    133077

[[97022  2929]
 [26494  6632]]
Test Results

             precision    recall  f1-score   support

        0.0       0.77      0.95      0.85     33430
        1.0       0.45      0.13      0.20     10930

avg / total       0.69      0.75      0.69     44360

[[31715  1715]
 [ 9550  1380]]
